In [36]:
## Import the required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import warnings
# import kagglehub

warnings.filterwarnings('ignore')

In [37]:


# url = "https://docs.google.com/spreadsheets/d/1rogG8b-5v_bdoVSuG8ngg5l7WirMgNOWV7GGb134P2Q/export?format=csv"

df = pd.read_excel("/Applications/Machine Learning/Spam detection/spam_dataset.xlsx")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [38]:
df.shape

(16533, 5)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16533 entries, 0 to 16532
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          16533 non-null  object
 1   v2          16517 non-null  object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 645.9+ KB


## Data Cleaning

In [40]:
df = df[['v1', 'v2']]
df = df.rename(columns={'v1': 'target', 'v2': 'text'})
df.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [41]:
df = df.replace({'ham': 0, 'spam': 1})
df.head()

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [42]:
df.duplicated().sum()

np.int64(5821)

In [43]:
df = df.drop_duplicates(keep='first')
df = df.reset_index(drop=True)
df.shape

(10712, 2)

In [44]:
df.isnull().sum()

target    0
text      1
dtype: int64

In [45]:
df = df.dropna()

In [46]:
df.isnull().sum()

target    0
text      0
dtype: int64

In [47]:
df.value_counts('target')

target
0    8237
1    2474
Name: count, dtype: int64

In [48]:
## adding new row

df.loc[len(df)] = {'target' : 1, 'text' : "Dear user, due to unusual login attempts, we have temporarily locked your account. Please verify your identity within 24 hours to avoid suspension. Click here: secure-login.com/verify"}

In [49]:
df.tail()

,target,text
10707,1,Hey little one! Exciting news! Mama and baby a...
10708,1,Amazing DATA deals on your Pulse Plan today! D...
10709,1,Special offer just for you! Get 1GB @15 bob va...
10710,1,NEW ARRIVAL - JUNE 23RD Dresses @ 300; Kondel...
10711,1,"Dear user, due to unusual login attempts, we h..."


## Preprocess the Text

In [50]:
# from typing_extensions import Text
import re

## r'\d+' - > removes the digits
## r'[^\w\s]'  -> removes digits and punctuations
## '' -> removes the empty string

def preprocess(text):
  if text is not None:

    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)
    
    text = text.strip()
  else:
    text = ''
  return text

In [51]:
df['text'] = df['text'].apply(preprocess)

In [52]:
df.tail()

,target,text
10707,1,hey little one exciting news mama and baby are...
10708,1,amazing data deals on your pulse plan today di...
10709,1,special offer just for you get 1gb 15 bob vali...
10710,1,new arrival june 23rd dresses 300 kondele b...
10711,1,dear user due to unusual login attempts we hav...


## Split Data into Training and Testing Sets


In [53]:

x = df['text']
y = df['target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [54]:
## Convert Text into TF-IDF Features
## TF-IDF is a NLP technique which converts text into data

vector = TfidfVectorizer(ngram_range=(1, 2))
x_train_tfidf = vector.fit_transform(x_train)
x_test_tfidf = vector.transform(x_test)

In [55]:
x_test_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 202311 stored elements and shape (2143, 305567)>

In [56]:
model = SVC(probability=True)
model.fit(x_train_tfidf, y_train)

Exception ignored in: <function ResourceTracker.__del__ at 0x106ed3880>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x1048af880>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/m

SVC(probability=True)

In [57]:
y_pred = model.predict(x_test_tfidf)

In [58]:
accuracy = accuracy_score(y_test, y_pred)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print(f"Model Accuracy: {accuracy*100:.2f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[1659   10]
 [  85  389]]
Model Accuracy: 95.57

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1669
           1       0.97      0.82      0.89       474

    accuracy                           0.96      2143
   macro avg       0.96      0.91      0.93      2143
weighted avg       0.96      0.96      0.95      2143



## Hyperparameter Tunning

In [63]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "C": [10],
    "kernel": ["linear"],
    "gamma": ["scale"]
}

svc = SVC(probability=True)

In [64]:
## svc is model you are tuning
## after finding the best combination it will refit the model on entire training data using those best parameters

grid = GridSearchCV(svc, param_grid, refit=True, cv = 5, scoring="accuracy", n_jobs = -1)
grid.fit(x_train_tfidf, y_train)

Exception ignored in: <function ResourceTracker.__del__ at 0x10458f880>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x106b8b880>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/m

GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10], 'gamma': ['scale'], 'kernel': ['linear']},
             scoring='accuracy')

In [65]:
y_pred_grid = grid.predict(x_test_tfidf)

In [66]:
print("Best Parameters:", grid.best_params_)
print("Best Score:", grid.best_score_)

Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Best Score: 0.9505134393385472


In [67]:
accuracy = accuracy_score(y_test, y_pred_grid)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_grid))

print(f"Model Accuracy: {accuracy*100:.2f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_grid))

Confusion Matrix:
[[1653   16]
 [  39  435]]
Model Accuracy: 97.43

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1669
           1       0.96      0.92      0.94       474

    accuracy                           0.97      2143
   macro avg       0.97      0.95      0.96      2143
weighted avg       0.97      0.97      0.97      2143



## Hyperparameter Tunning with RandomCV

In [68]:
# from sklearn.model_selection import RandomizedSearchCV

# params = {
#     "C": [0.1, 1, 10],
#     "kernel": ["linear", "rbf"],
#     "gamma": ["scale", "auto"]
# }


In [69]:
# randomcv = RandomizedSearchCV(svc, param_distributions=params, n_iter=10, cv=5, scoring="accuracy", n_jobs=5)
# randomcv.fit(x_train_tfidf, y_train)

In [70]:
# y_pred_random = randomcv.predict(x_test_tfidf)

In [71]:
# print("Best Parameters:", randomcv.best_params_)
# print("Best Score:", randomcv.best_score_)

In [72]:
# accuracy = accuracy_score(y_test, y_pred_random)

# print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_pred_random))

# print(f"Model Accuracy: {accuracy*100:.2f}")

# print("\nClassification Report:")
# print(classification_report(y_test, y_pred_random))

In [73]:
## Comparing both searches Model accuracy is better with randomcv


In [74]:
import joblib

joblib.dump(grid, "Spam_Detection.pkl")
joblib.dump(vector, "tfidf_vectorizer.pkl")

['tfidf_vectorizer.pkl']

Exception ignored in: <function ResourceTracker.__del__ at 0x106ee3880>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x104efb880>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/m